In [1]:
import pandas as pd
import numpy as np
!pip install torch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset


/Users/bnarayanan/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:


data_daily = pd.read_csv('data_daily.csv', parse_dates=['# Date'])
data_daily.set_index('# Date', inplace=True)

# Resample the data to monthly frequency and sum the receipts for each month
monthly_data = data_daily['Receipt_Count'].resample('M').sum()



monthly_data.to_csv('monthly_receipts_2021.csv')
monthly_data = pd.read_csv('monthly_receipts_2021.csv')
monthly_data = monthly_data.rename(columns={'# Date': 'Date'})
monthly_data['Date'] = pd.to_datetime(monthly_data['Date'])
monthly_data['Month'] = monthly_data['Date'].dt.month
print(monthly_data)


         Date  Receipt_Count  Month
0  2021-01-31      236736687      1
1  2021-02-28      220033460      2
2  2021-03-31      248608625      3
3  2021-04-30      250644830      4
4  2021-05-31      263151748      5
5  2021-06-30      260656840      6
6  2021-07-31      274776003      7
7  2021-08-31      283943231      8
8  2021-09-30      281146154      9
9  2021-10-31      295965185     10
10 2021-11-30      296085162     11
11 2021-12-31      309948684     12


In [3]:
X = monthly_data['Month'].values
y = monthly_data['Receipt_Count'].values

In [4]:
split_ratio = 0.8

split_index = int(len(X) * split_ratio)
X_train, y_train = X[:split_index], y[:split_index]
X_test, y_test = X[split_index:], y[split_index:]

print(X_train)

[1 2 3 4 5 6 7 8 9]


In [5]:
class ReceiptPredictionModel(nn.Module):
    def __init__(self):
        super(ReceiptPredictionModel, self).__init__()
        self.fc1 = nn.Linear(1, 100)
        self.dropout1 = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(100, 100)
        self.dropout2 = nn.Dropout(p=0.5)
        self.fc3 = nn.Linear(100, 50)
        self.dropout3 = nn.Dropout(p=0.5)
        self.fc4 = nn.Linear(50, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)
        x = torch.relu(self.fc3(x))
        x = self.dropout3(x)
        x = self.fc4(x)
        return x
model = ReceiptPredictionModel()

In [6]:
criterion = nn.MSELoss()
weight_decay = 0.001
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=weight_decay)

num_epochs = 1000

for epoch in range(num_epochs):
    # Convert data to PyTorch tensors
    inputs = torch.Tensor(X_train).view(-1, 1)
    labels = torch.Tensor(y_train).view(-1, 1)

    # Zero the gradients
    optimizer.zero_grad()

    # Forward pass
    outputs = model(inputs)

    # Calculate the loss
    loss = criterion(outputs, labels)

    # Backpropagation and optimization
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

print('Training complete')

# Make predictions for 2022
months_2022 = np.arange(13, 25)
inputs_2022 = torch.Tensor(months_2022).view(-1, 1)
predictions_2022 = model(inputs_2022).detach().numpy()

print("Predictions for 2022:", predictions_2022)

Epoch [100/1000], Loss: 66827127030284288.0000
Epoch [200/1000], Loss: 66819786931175424.0000
Epoch [300/1000], Loss: 66782678413737984.0000
Epoch [400/1000], Loss: 66640970262773760.0000
Epoch [500/1000], Loss: 66288340562870272.0000
Epoch [600/1000], Loss: 65779060520779776.0000
Epoch [700/1000], Loss: 65087386102530048.0000
Epoch [800/1000], Loss: 63737808593879040.0000
Epoch [900/1000], Loss: 61427056059088896.0000
Epoch [1000/1000], Loss: 59026220880232448.0000
Training complete
Predictions for 2022: [[21073060.]
 [44204048.]
 [24760558.]
 [31366204.]
 [70750608.]
 [44231452.]
 [60572504.]
 [49029184.]
 [30043462.]
 [49244688.]
 [51960912.]
 [90987784.]]
